# Practice - Shinydashboard Library

We can create dashboards by using the Shiny functions we have seen previously. 
We can also use the  ```shinydashboard``` library to help us easily create good looking dashboards. 

**Note that the ```shinydashboard``` library has its own functions to create layouts,
and it's usually good practice not to mix these functions with the default Shiny functions we have seen so far.
You can still create dashboards without using this library; 
it's optional and provides a few nice layouts in case you want to use them. **


A ```shinydashboard```-style dashboard has three parts: a header, a sidebar, and a body. 
Here is the most minimal possible UI for a dashboard page.

---
```R
# app.R 
library(shiny)
library(shinydashboard)

ui <- dashboardPage(
  dashboardHeader(),
  dashboardSidebar(),
  dashboardBody()
)

server <- function(input, output) { }

shinyApp(ui=ui, server=server)
```
---

This creates a blank dashboard. 
** Copy this to app.R file under ```module2-dashboard1``` and deploy it** to see how it differs from the usual Shiny app look. 

In [ ]:
# RUN THIS TO DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module2-dashboard1" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

To add components, we can use the ```box``` functions inside fluidRows.

---

```
# app.R
library(shiny)
library(shinydashboard)

ui <- dashboardPage(
  dashboardHeader(title = "Basic dashboard"),
  dashboardSidebar(),
  dashboardBody(
    # Boxes need to be put in a row (or column)
    fluidRow(
      box(plotOutput("plot1", height = 250)),

      box(
        title = "Controls",
        sliderInput("slider", "Number of observations:", 1, 100, 50)
      )
    )
  )
)

server <- function(input, output) {
  set.seed(122)
  histdata <- rnorm(500)

  output$plot1 <- renderPlot({
    data <- histdata[seq_len(input$slider)]
    hist(data)
  })
}

shinyApp(ui, server)

```
---

This code creates the following dashboard:

<img src="../images/basicd.png">

**Modify your app.R in ```module2-dashboard1``` to copy and deploy the code above ** and check if it looks similar to this picture. 


Next, we can add content to the sidebar. 
For this example we’ll add menu items that behave like tabs.
These function similarly to Shiny’s tabPanels: 
when you click on one menu item, 
it shows a different set of content in the main body.

There are two parts that need to be done.
First, you need to add menuItems to the sidebar with appropriate tabNames.


---
```R
# Sidebar content
  dashboardSidebar(
    sidebarMenu(
      menuItem("Dashboard", tabName = "dashboard", icon = icon("dashboard")),
      menuItem("Widgets", tabName = "widgets", icon = icon("th"))
    )
  )
```
---


In the body, add tabItems with corresponding values for tabName:


---
```R
# Body content
  dashboardBody(
    tabItems(
      # First tab content
      tabItem(tabName = "dashboard",
        fluidRow(
          box(plotOutput("plot1", height = 250)),

          box(
            title = "Controls",
            sliderInput("slider", "Number of observations:", 1, 100, 50)
          )
        )
      ),

      # Second tab content
      tabItem(tabName = "widgets",
        h2("Widgets tab content")
      )
    )
  )
```
---


This is how it looks like: 


<img src="../images/basic-tabs-1.png">


The HTML tag functions in Shiny, like div() and p() return objects that can be rendered as HTML. 
Some functions return more complex HTML fragments, 
and they insulate you from having to know all the ins and outs of the HTML required to create things like a text input or a sidebar.

The UI for a Shiny app is built out of these pieces of HTML.
The _shinydashboard_ package provides a set of functions designed to create HTML that will generate a dashboard. 

### Boxes

Boxes are the main building blocks of dashboard pages. 
A basic box can be created with the ```box()``` function, 
and the contents of the box can be any Shiny UI content.

In a typical dashboard, these boxes would be placed inside a `fluidRow()`:

---
```R
# This is just the body component of a dashboard
dashboardBody(
  fluidRow(
    box(plotOutput("plot1")),
    
    box(
      "Box content here", br(), "More box content",
      sliderInput("slider", "Slider input:", 1, 100, 50),
      textInput("text", "Text input:")
    )
  )
)
```



---

Boxes can have titles and header bar colors with the title and status options. 

---
```R
box(title = "Histogram", status = "primary", plotOutput("plot2", height = 250)),

box(
  title = "Inputs", status = "warning",
  "Box content here", br(), "More box content",
  sliderInput("slider", "Slider input:", 1, 100, 50),
  textInput("text", "Text input:")
)
```
---


<img src="../images/body-boxes-header.png">


### tabBox

If you want a box to have tabs for displaying different sets of content, you can use a tabBox. 
A tabBox is similar to a tabsetPanel from Shiny in that it takes tabPanels as inputs, 
allows you to choose which tab is selected, and can be assigned an id. 
If id is present, you can access which tab is selected in the server code. 
In the example below, the third tabBox displays which tab is selected in the first tabBox because we have access to the ```id``` of the first box. 


In [ ]:
# app.R
library(shiny)
library(shinydashboard)
# We do not have to put all the code inside the UI component; we can define 
# components outside and then use them to build the UI. 
# This is the definition of the `body` component
body <- dashboardBody(
  fluidRow(
    tabBox(
      title = "First tabBox",
      # The id lets us use input$tabset1 on the server to find the current tab
      id = "tabset1", height = "250px",
      tabPanel("Tab1", "First tab content"),
      tabPanel("Tab2", "Tab content 2")
    ),
    tabBox(
      side = "right", height = "250px",
      selected = "Tab3",
      tabPanel("Tab1", "Tab content 1"),
      tabPanel("Tab2", "Tab content 2"),
      tabPanel("Tab3", "Note that when side=right, the tab order is reversed.")
    )
  ),
  fluidRow(
    tabBox(
      # Title can include an icon
      title = tagList(shiny::icon("gear"), "tabBox status"),
      tabPanel("Tab1",
        "Currently selected tab from first box:",
        verbatimTextOutput("tabset1Selected")
      ),
      tabPanel("Tab2", "Tab content 2")
    )
  )
)

# This is UI, it has a clean look because we defined body up above. 
ui <- dashboardPage(
    dashboardHeader(title = "tabBoxes"),
    dashboardSidebar(),
    body
  )

# This is server
server <- function(input, output) {
    # The currently selected tab from the first box
    output$tabset1Selected <- renderText({
      input$tabset1
    })
  }

shinyApp(ui=ui, server=server)

** Copy and paste the above code to a file named app.R under module2-dashboard2 and deploy it to the Shiny server.** You should see a layout similar to the following: 


<img src="../images/body-tabboxes.png">


In [ ]:
# RUN THIS TO DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module2-dashboard2" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

Let's recreate the example from the ```L2_Practice_Layout``` notebook using the ```shinydashboard``` library. 
We will use ```tabBox``` function to create the plot and the data tabs, and ```box``` function to create the selection widgets. 
Note that we have to define the **width of the boxes** summing up to 12 just like we did with ```column``` function in Shiny. 

In [ ]:
# app.R
library(shiny)
library(shinydashboard)
library(ggplot2)

# these operations need to be done only once
# read the data set 
popdata = read.csv("../module2-pop/pop_proj.csv") # data set is in the same folder as the app. 

# we need these lists to automatically populate the choices in the UI. 
sex_list <- c("All", levels(popdata$sex))
age_list <- c("All", levels(popdata$age))

body <- dashboardBody(
    # first create a row 
    fluidRow(
     # now a tabBox with two panels    
     tabBox(
      title = "US Population",
      id = "proj", width=12, #full width of the screen 
         
      # this is the panel with the plot and the selections    
      tabPanel("Projection", 
        
        plotOutput("ggp"),
        box(title = tagList(shiny::icon("gear"), "Selections"), width=12,                       
      
          column(6,
                selectInput("sex_in", 
                  label = em("Sex:"),
                  choices = sex_list,
                  selected = "All") #default
          ),
          column(6,
                selectInput("age_in", 
                  label = em("Age bracket:"),
                  choices = age_list,
                  selected = "All") #default
          )
        )               
      ),
         
      # this is the data panel    
      tabPanel("Data", dataTableOutput("table"))      
    )
             
  )
)

# This is UI, it has a clean look because we defined body up above. 
ui <- dashboardPage(
    dashboardHeader(title = "popVis"),
    dashboardSidebar(helpText("Projections of the population by sex and selected age groups for US: 2015-2030")),
    body
  )

# This is server function is the same as in the layout practice. 
server <- function(input, output) {

     output$ggp <- renderPlot({ 
      
      if (input$age_in != "All") 
      {
        newdata <- subset(popdata, age == input$age_in)
      } else { 
          
        newdata <- popdata
      }
   
      if (input$sex_in == "All") 
      {
          p <- ggplot(data=newdata, aes_string(x="year", y="pop")) 

      } else {
          
          p <- ggplot(data=subset(newdata, sex == input$sex_in), aes_string(x="year", y="pop"))           
      }
      
      p <- p + geom_bar(stat="identity") + theme_minimal() + xlab("Years") + ylab("Population Projection") + ylim(0,300000) 
      print(p)
  })
 
  output$table <- renderDataTable(popdata)    
}

shinyApp(ui=ui, server=server)

In [ ]:
# RUN THIS TO DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module2-pop2" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

# Save your notebook